# Autonomous Delivery Robot

### Brief Explanation
###### ROBOT == TURQUOISE COLOR 
###### DELIVERY POINTS == RED COLOR 
###### CAR == ORANGE COLOR 
###### BUILDINGS == GRAY COLOR 
###### TREES == DRAK GREEN COLOR 
###### HOUSE == BROWN COLOR 

##### Using A* Search our Autonomous Delivery Robot navigate through an area in the city to deliver packages to designated locations avoiding obstacles .


In [23]:
import pygame
import heapq
import random
import time
import math

In [24]:
# colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
DARK_GREEN = (0, 100, 0)
ORANGE = (255, 165, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
TURQUOISE = (64, 224, 208)
BROWN = (165, 42, 42)
GRAY = (128, 128, 128)

GRID_SIZE = 15
WINDOW_SIZE = [500, 500]
CELL_SIZE = WINDOW_SIZE[0] // GRID_SIZE
OBSTACLE_PROBABILITY = 0.12  

In [25]:
# Random delivery locations
DELIVERY_LOCATIONS = []
for _ in range(5):
    x = random.randint(0, GRID_SIZE - 1)
    y = random.randint(0, GRID_SIZE - 1)
    DELIVERY_LOCATIONS.append((x, y))

# Initialize pygame
pygame.init()

# Set up the window
window = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("Autonomous Delivery Robot")

#grid
grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]



In [26]:
# Generate obstacles and cars
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        if random.random() < OBSTACLE_PROBABILITY:
            grid[i][j] = random.choice([2, 3, 4])  # Representing obstacles
        elif random.random() < 0.07:  # 7% chance of car       
            grid[i][j] = 5  # Representing cars
            
# Set the starting location
grid[0][0] = 0  # initial location
for loc in DELIVERY_LOCATIONS:
    grid[loc[0]][loc[1]] = 0


##### A* Search is better than Best-First Search for the scenario of an autonomous delivery robot navigating through a city environment because it guarantees optimality and efficiency by combining actual and estimated costs while ensuring completeness.

In [27]:
#  heuristic function uding EUCLIDEAN DISTANCE
def heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

#  A* algorithm
def astar(start, goal):
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while frontier:
        current = heapq.heappop(frontier)[1]

        if current == goal:
            path = []
            while current != start:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        for next in [(current[0] + 1, current[1]), (current[0] - 1, current[1]),
                     (current[0], current[1] + 1), (current[0], current[1] - 1)]:
            if 0 <= next[0] < GRID_SIZE and 0 <= next[1] < GRID_SIZE and grid[next[0]][next[1]] == 0:
                new_cost = cost_so_far[current] + 1
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    cost_so_far[next] = new_cost
                    priority = new_cost + heuristic(next, goal)
                    heapq.heappush(frontier, (priority, next))
                    came_from[next] = current

    return []



In [28]:
def main():
    global DELIVERY_LOCATIONS
    running = True
    path = []
    start = (0, 0)

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Clearing window
        window.fill(WHITE)

        # Draw the grid with black borders
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                rect = pygame.Rect(j * CELL_SIZE, i * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                
                # Draw black border
                pygame.draw.rect(window, BLACK, rect, 1)
                
                # Fill grid cells with colors
                if grid[i][j] == 1:  # Normal grid
                    pygame.draw.rect(window, WHITE, rect)
                elif grid[i][j] == 2:  # Trees
                    pygame.draw.rect(window, DARK_GREEN, rect)
                elif grid[i][j] == 3:  # Buildings
                    pygame.draw.rect(window, GRAY, rect)
                elif grid[i][j] == 4:  # Houses
                    pygame.draw.rect(window, BROWN, rect)
                elif grid[i][j] == 5:  # Cars
                    pygame.draw.rect(window, ORANGE, rect)

        # Draw the delivery locations as circles
        for loc in DELIVERY_LOCATIONS:
            pygame.draw.circle(window, RED, (loc[1] * CELL_SIZE + CELL_SIZE // 2, loc[0] * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 2)

        # Draw the robot as a rectangle
        robot_rect = pygame.Rect(start[1] * CELL_SIZE, start[0] * CELL_SIZE, CELL_SIZE, CELL_SIZE)
        pygame.draw.rect(window, TURQUOISE, robot_rect)

        # Update the display
        pygame.display.flip()

        # Deliver items
        if DELIVERY_LOCATIONS:
            current_goal = min(DELIVERY_LOCATIONS, key=lambda loc: heuristic(start, loc))
            path = astar(start, current_goal)
            if path:
                while path:
                    next_step = path.pop(0)
                    start = next_step
                    robot_rect = pygame.Rect(start[1] * CELL_SIZE, start[0] * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                    pygame.draw.rect(window, TURQUOISE, robot_rect)
                    pygame.display.flip()
                    time.sleep(0.4)
                DELIVERY_LOCATIONS.remove(current_goal)
                # If all deliveries are made, stop the robot
                if not DELIVERY_LOCATIONS:
                    break
        else:
            # Return to initial location
            path_back = astar(start, (0, 0))
            if path_back:
                while path_back:
                    next_step = path_back.pop(0)
                    start = next_step
                    robot_rect = pygame.Rect(start[1] * CELL_SIZE, start[0] * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                    pygame.draw.rect(window, RED, robot_rect)
                    pygame.display.flip()
                    time.sleep(0.3)
                    if not path_back:
                        break
            break  # Stop once all deliveries are completed

    pygame.quit()


In [29]:
if __name__ == "__main__":
    main() 